# EQTYahoo:
---

EQTYahoo interacts with Yahoo Finance APIs to fetch market, financial, options, and fundamental data, storing results in MongoDB. It requires Python 3.10+ and a running MongoDB instance.  



#### MongoDB Installation:

1. Download MongoDB Community Server from [https://www.mongodb.com/try/download/community](https://www.mongodb.com/try/download/community).  
2. Run the installer, pick Complete setup, and start the MongoDB service.  
3. (Optional) Install MongoDB Compass for a graphical interface.

Ensure the MongoDB service is running before using EQTYahoo.

---

In [3]:
import EQTYahoo
import pandas
pandas.set_option('display.max_columns', None)

## <u>Credentials Module</u>:

Located in Credentials.py, its role is to manage authentication details (Headers, Cookies, Crumb):

In [4]:

C = EQTYahoo.Credentials.Get()
print(C.Headers)  # Shows the custom headers

{'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}


This ensures valid requests to Yahoo endpoints.



## <u>Quotes Module</u>:

Located in Quotes.py, it retrieves historical data for one or multiple tickers:


In [35]:
EQTYahoo.Quotes.HistoricalData('AAPL', '1d', "2024-01-01", "2025-01-01").head(5) # Get historical data for a Single Ticker

AAPL                                      \
                           Open        High         Low       Close   
Date                                                                  
2024-01-02 18:30:00  187.149994  188.440002  183.889999  185.639999   
2024-01-03 18:30:00  184.220001  185.880005  183.429993  184.250000   
2024-01-04 18:30:00  182.149994  183.089996  180.880005  181.910004   
2024-01-05 18:30:00  181.990005  182.759995  180.169998  181.179993   
2024-01-08 18:30:00  182.089996  185.600006  181.500000  185.559998   

                                              
                    Adjusted Close    Volume  
Date                                          
2024-01-02 18:30:00     184.734970  82488700  
2024-01-03 18:30:00     183.351746  58414500  
2024-01-04 18:30:00     181.023178  71983600  
2024-01-05 18:30:00     180.296707  62303300  
2024-01-08 18:30:00     184.655365  59144500

In [36]:
EQTYahoo.Quotes.HistoricalData(['AAPL', 'MSFT'], '1d', "2024-01-01", "2025-01-01").head(5) # Get historical data for Multiple Tickers

AAPL                                      \
                           Open        High         Low       Close   
Date                                                                  
2024-01-02 18:30:00  187.149994  188.440002  183.889999  185.639999   
2024-01-03 18:30:00  184.220001  185.880005  183.429993  184.250000   
2024-01-04 18:30:00  182.149994  183.089996  180.880005  181.910004   
2024-01-05 18:30:00  181.990005  182.759995  180.169998  181.179993   
2024-01-08 18:30:00  182.089996  185.600006  181.500000  185.559998   

                                                    MSFT              \
                    Adjusted Close    Volume        Open        High   
Date                                                                   
2024-01-02 18:30:00     184.734970  82488700  373.859985  375.899994   
2024-01-03 18:30:00     183.351746  58414500  369.010010  373.260010   
2024-01-04 18:30:00     181.023178  71983600  370.670013  373.100006   
2024-01-05 18:30:00     180.296707  62303300  368.970001  372.059998   
2024-01-08 18:30:00     184.655365  59144500  369.299988  375.200012   

                                                                      
                            Low       Close Adjusted Close    Volume  
Date                                                                  
2024-01-02 18:30:00  366.769989  370.869995     368.117249  25258600  
2024-01-03 18:30:00  368.510010  370.600006     367.849274  23083500  
2024-01-04 18:30:00  367.170013  367.940002     365.208984  20901500  
2024-01-05 18:30:00  366.500000  367.750000     365.020416  20987000  
2024-01-08 18:30:00  369.010010  374.690002     371.908905  23134000

Data is organized in MongoDB, ensuring partial updates if some dates are missing. For example: You previously asked data from 2020-01-01 to 2025-01-01. You now query data from 2015-01-01 to to 2025-01-25. Data will be downloaded only for missing parts. 

## <u>Screener Module</u>:
In Screener.py, it uses region mapping plus other criteria (sector, industry, etc.).
First, build the screener database (one-time):

In [ ]:
EQTYahoo.Screener.EquitiesDB() # ~60s to fetch ~58k tickers

Once it's done, you'll be able to query the Data using the `Equities` function:

In [37]:
US5 = EQTYahoo.Screener.Equities(Region='United States').sort_values(by='Market Cap', ascending=False).head(5).drop(columns="_id") # Get the Top 5 US Equities by Market Cap
display(US5)

,Region,Ticker,Company Name,Sector,Price,Change %,Change,52W Low,52W High,Volume,Avg Vol (3m),Market Capitalization,Industry,Last Update,Language,Quote Type,Type Disp,Quote Source Name,Triggerable,Custom Price Alert Confidence,Currency,Corporate Actions,Pre Market Time,Regular Market Time,Exchange,Message Board Id,Exchange Timezone Name,Exchange Timezone Short Name,Gmt Off Set Milliseconds,Market,Esg Populated,Regular Market Change Percent,Regular Market Price,Short Name,Long Name,Market State,Has Pre Post Market Data,First Trade Date Milliseconds,Price Hint,Pre Market Change,Pre Market Change Percent,Pre Market Price,Regular Market Change,Regular Market Day High,Regular Market Day Range,Regular Market Day Low,Regular Market Volume,Regular Market Previous Close,Bid,Ask,Bid Size,Ask Size,Full Exchange Name,Financial Currency,Regular Market Open,Average Daily Volume3 Month,Average Daily Volume10 Day,Fifty Two Week Low Change,Fifty Two Week Low Change Percent,Fifty Two Week Range,Fifty Two Week High Change,Fifty Two Week High Change Percent,Fifty Two Week Low,Fifty Two Week High,Fifty Two Week Change Percent,Earnings Timestamp,Earnings Timestamp Start,Earnings Timestamp End,Earnings Call Timestamp Start,Earnings Call Timestamp End,Is Earnings Date Estimate,Trailing Annual Dividend Rate,Trailing Annual Dividend Yield,Eps Trailing Twelve Months,Eps Forward,Eps Current Year,Price Eps Current Year,Shares Outstanding,Book Value,Fifty Day Average,Fifty Day Average Change,Fifty Day Average Change Percent,Two Hundred Day Average,Two Hundred Day Average Change,Two Hundred Day Average Change Percent,Market Cap,Forward P E,Price To Book,Source Interval,Exchange Data Delayed By,Ipo Expected Date,Prev Name,Name Change Date,Average Analyst Rating,Tradeable,Crypto Tradeable,Display Name,P/E Ratio,Dividend Date,Trailing P E,Dividend Rate,Dividend Yield,logoUrl,Prev Exchange,Exchange Transfer Date,Underlying Symbol,Open Interest
3726,United States,AAPL,Apple Inc.,Technology,238.35,3.693555,8.490006,164.08,260.10,58530547.0,4.964993e+07,3.602851e+12,Consumer Electronics,1.738105e+09,en-US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,[],1.738144e+09,1.738098e+09,NMS,finmb_24937,America/New_York,EST,-18000000.0,us_market,False,3.654396,238.26,Apple Inc.,Apple Inc.,PRE,True,3.454794e+11,2.0,-1.069992,-0.449086,237.19,8.399994,240.19,230.88 - 240.19,230.880,75122318.0,229.86,223.95,251.98,1.0,1.0,NasdaqGS,USD,230.880,49649926.0,66714100.0,74.179990,0.452097,164.08 - 260.1,-21.840012,-0.083968,164.08,260.10,29.208242,1.738271e+09,1.738271e+09,1.738271e+09,1.738274e+09,1.738274e+09,False,0.98,0.004263,6.08,8.31,7.39843,32.204130,1.503790e+10,3.767,239.5358,-1.275803,-0.005326,218.751050,19.508942,0.089183,3.582930e+12,28.671478,63.249268,15.0,0.0,NaN,NaN,NaN,2.0 - Buy,False,False,Apple,37.026315,1.731542e+09,39.187500,1.00,0.42,https://s.yimg.com/lb/brands/150x150_apple.png,NaN,NaN,NaN,NaN
30,United States,MSFT,Microsoft Corporation,Technology,447.20,2.908693,12.640015,385.58,468.35,18149904.0,2.193492e+07,3.324878e+12,Software—Infrastructure,1.738105e+09,en-US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,"[{'header': 'Dividend', 'message': 'MSFT annou...",1.738144e+09,1.738098e+09,NMS,finmb_21835,America/New_York,EST,-18000000.0,us_market,False,2.908693,447.20,Microsoft Corporation,Microsoft Corporation,PRE,True,5.111082e+11,2.0,-0.830017,-0.185603,446.37,12.640015,448.38,431.425 - 448.38,431.425,23138065.0,434.56,420.20,465.00,1.0,3.0,NasdaqGS,USD,434.600,21934923.0,22501910.0,61.620026,0.159811,385.58 - 468.35,-21.149994,-0.045159,385.58,468.35,12.480509,1.738184e+09,1.738184e+09,1.738184e+09,1.738190e+09,1.738190e+09,False,3.08,0.007088,12.12,14.95,13.01874,34.350483,7.434880e+09,38.693,431.2496,15.950409,0.036986,425.681300,21.518707,0.050551,3.324879e+12,29.913044,11.557646,15.0,0.0,NaN,NaN,NaN,1.4 - Strong Buy,False,False,Microsoft,35.111475,1.741824e+09,36.897690,3.32,0.74,https://s.yimg.com/lb/brands/150x150_microsoft...,NaN,NaN,NaN

In [38]:
FR5 = EQTYahoo.Screener.Equities(Region="France", Currency='EUR', Exchange='PAR').sort_values(by='Market Cap', ascending=False).head(5).drop(columns="_id") # Get the Top 5 French Equities by Market Cap
display(FR5)

,Region,Ticker,Company Name,Sector,Price,Change %,Change,52W Low,52W High,Volume,Avg Vol (3m),Market Capitalization,P/E Ratio,Industry,Last Update,Language,Quote Type,Type Disp,Quote Source Name,Triggerable,Custom Price Alert Confidence,Currency,Corporate Actions,Regular Market Time,Exchange,Message Board Id,Exchange Timezone Name,Regular Market Change Percent,Regular Market Price,Has Pre Post Market Data,First Trade Date Milliseconds,Price Hint,Short Name,Long Name,Market State,Regular Market Change,Regular Market Day High,Regular Market Day Range,Regular Market Day Low,Regular Market Volume,Regular Market Previous Close,Bid,Ask,Bid Size,Ask Size,Full Exchange Name,Financial Currency,Regular Market Open,Average Daily Volume3 Month,Average Daily Volume10 Day,Fifty Two Week Low Change,Fifty Two Week Low Change Percent,Fifty Two Week Range,Fifty Two Week High Change,Fifty Two Week High Change Percent,Fifty Two Week Low,Fifty Two Week High,Fifty Two Week Change Percent,Earnings Timestamp,Earnings Timestamp Start,Earnings Timestamp End,Earnings Call Timestamp Start,Earnings Call Timestamp End,Is Earnings Date Estimate,Trailing Annual Dividend Rate,Trailing Annual Dividend Yield,Eps Trailing Twelve Months,Eps Forward,Eps Current Year,Price Eps Current Year,Shares Outstanding,Book Value,Fifty Day Average,Fifty Day Average Change,Fifty Day Average Change Percent,Two Hundred Day Average,Two Hundred Day Average Change,Two Hundred Day Average Change Percent,Market Cap,Forward P E,Price To Book,Source Interval,Exchange Data Delayed By,Average Analyst Rating,Tradeable,Crypto Tradeable,Exchange Timezone Short Name,Gmt Off Set Milliseconds,Market,Esg Populated,Trailing P E,Prev Name,Name Change Date,Dividend Rate,Dividend Yield,Dividend Date,Ipo Expected Date
27,France,MC.PA,"LVMH Moët Hennessy - Louis Vuitton, Société Eu...",Consumer Cyclical,704.80,-6.101784,-45.799988,565.40,886.40,323261.0,4.661267e+05,3.523915e+11,20.546532,Luxury Goods,1.738105e+09,en-US,EQUITY,Equity,Delayed Quote,False,LOW,EUR,[],1738143010,PAR,finmb_195488,Europe/Paris,-6.102597,704.80,False,946886400000,2,LVMH,"LVMH Moët Hennessy - Louis Vuitton, Société Eu...",REGULAR,-45.799988,722.20,700.0 - 722.2,700.00,325016,750.6,0.0,0.0,0.0,0.0,Paris,EUR,720.50,466126,626800,139.399960,0.246551,565.4 - 886.4,-181.600040,-0.204874,565.40,886.40,-2.430195,1.738076e+09,1.738076e+09,1.738076e+09,1.738084e+09,1.738084e+09,False,13.0,0.017319,27.92,29.09,28.04615,25.130009,499372992,129.728,636.052,68.747986,0.108085,671.39700,33.403015,0.049752,353927299072,24.228256,5.432906,15,15,1.9 - Buy,False,False,CET,3600000,fr_market,False,25.243553,NaN,NaN,13.0,1.73,NaN,NaN
376,France,RMS.PA,Hermès International Société en commandite par...,Consumer Cyclical,2648.00,-0.526118,-14.000000,1888.00,2708.00,15007.0,5.773600e+04,2.791257e+11,47.195430,Luxury Goods,1.738105e+09,en-US,EQUITY,Equity,Delayed Quote,False,LOW,EUR,[],1738143004,PAR,finmb_881746,Europe/Paris,-0.525920,2648.00,False,946886400000,2,HERMES INTL,Hermès International Société en commandite par...,REGULAR,-14.000000,2655.00,2621.0 - 2655.0,2621.00,15042,2662.0,0.0,0.0,0.0,0.0,Paris,EUR,2630.00,57736,65237,760.000000,0.402542,1888.0 - 2708.0,-60.000000,-0.022157,1888.00,2708.00,35.968950,1.739516e+09,1.739516e+09,1.739516e+09,1.721923e+09,1.721923e+09,False,11.5,0.004320,42.33,47.68,42.90047,61.724266,104836000,143.721,2285.450,362.550050,0.158634,2162.62000,485.379880,0.224441,277605711872,55.536910,18.424587,15,15,2.1 - Buy,False,False,CET,3600000,fr_market,False,62.556103,NaN,NaN,15.0,0.56,NaN,NaN
63,France,OR.PA,L'Oréal S.A.,Consumer Defensive,357.25,-2.815562,-10.350006,316.30,461.85,90471.0,4.292350e+05,1.906475e+11,27.045156,Household & Personal Products,1.738105e+09,en-US,EQUITY,Equity,Delayed Quote,False,LOW,EUR,[],1738143011,PAR,finmb_874249,Europe/Paris,-2.734327,357.55,False,946886400000,2,L'OREAL,L'Oréal S.A.,REGULAR,-10.050018,363.20,356.05 - 363.2,356.05,91982,367.6,0.0,0.0,0.0,0.0,Paris,EUR,362.00,429234,478

Filters include: 

```
Region: Unknown | None = None,
Sector: Unknown | None = None,
Industry: Unknown | None = None,
Market: Unknown | None = None,
Currency: Unknown | None = None,
Exchange: Unknown | None = None,
FullExchangeName: Unknown | None = None,
MinMarketCap: Unknown | None = None,
MaxMarketCap: Unknown | None = None,
MinVolume: Unknown | None = None,
MaxVolume: Unknown | None = None
```

## <u>Information Module</u>:

Informations.py fetches detailed company info and fundamentals:



In [7]:
dir(EQTYahoo.Informations)

['AssetProfile',
 'BalanceSheetHistory',
 'BalanceSheetHistoryQuarterly',
 'CalendarEvents',
 'CashflowStatementHistory',
 'CashflowStatementHistoryQuarterly',
 'CompleteInformations',
 'Cookies',
 'Credentials',
 'Crumb',
 'DefaultKeyStatistics',
 'Earnings',
 'EarningsHistory',
 'EarningsTrend',
 'EsgScores',
 'FetchAndStore',
 'FinancialData',
 'FundOwnership',
 'FundPerformance',
 'FundProfile',
 'GetQuoteSummary',
 'Headers',
 'IncomeStatementHistory',
 'IncomeStatementHistoryQuarterly',
 'IndexTrend',
 'IndustryTrend',
 'InsiderHolders',
 'InsiderTransactions',
 'InstitutionOwnership',
 'MajorHoldersBreakdown',
 'NetSharePurchaseActivity',
 'Options',
 'PageViews',
 'Price',
 'QuoteSummary',
 'QuoteType',
 'RecommendationTrend',
 'RenameKey',
 'RenameKeysRecursively',
 'SecFilings',
 'SectorTrend',
 'StoreQuoteSummary',
 'SummaryDetail',
 'SummaryProfile',
 'TopHoldings',
 'UpgradeDowngradeHistory',
 'YFCredentials',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__lo

It pulls insights like balance sheet, insider transactions, and earnings, storing them in the Informations database. You can fetch all available information for a given ticker by using the `CompleteInformations` function:


In [8]:
EQTYahoo.Informations.CompleteInformations(FR5.iloc[0]['Ticker'])

No document found in MongoDB for MC.PA. Fetching from API...
Stored new data for MC.PA in MongoDB.


{'Asset Profile': {'Address1': '22, Avenue Montaigne',
  'City': 'Paris',
  'Zip': '75008',
  'Country': 'France',
  'Phone': '33 1 44 13 22 22',
  'Fax': '33 1 44 13 22 23',
  'Website': 'https://www.lvmh.com',
  'Industry': 'Luxury Goods',
  'Industry Key': 'luxury-goods',
  'Industry Disp': 'Luxury Goods',
  'Sector': 'Consumer Cyclical',
  'Sector Key': 'consumer-cyclical',
  'Sector Disp': 'Consumer Cyclical',
  'Long Business Summary': "LVMH Moët Hennessy - Louis Vuitton, Société Européenne operates as a luxury goods company worldwide. The company offers wines, and spirits under the domaine des Lambrays, Château d'Yquem, Dom Pérignon, Ruinart, Moët & Chandon, Hennessy, Veuve Clicquot, Château Galoupet, Ardbeg, Château Cheval Blanc, Glenmorangie, Krug, Mercier, Chandon, Joseph Phelps, Newton Vineyard, Cloudy Bay, Belvedere, Colgin Cellars,Terrazas de los Andes, Bodega Numanthia, Cheval des Andes, Woodinville, Ao Yun, Clos19, Volcan de mi Tierra, and Eminente brands; and fashion an

Every time you call a function from the `Information` module for a given ticker, it checks if the information is stored in the database. If so, it verifies whether the information is up to date. If not, it downloads the data and updates the database. The default update frequency is set to 24 hours. When you request information about that ticker, data from the database will be returned until the data is considered outdated. The main purpose behind this is to reduce execution times when processing a significant amount of data. The previous cell execution time was 0.3s; the following call takes 0.1s.


In [9]:
EQTYahoo.Informations.CompleteInformations(FR5.iloc[0]['Ticker'])

{'_id': ObjectId('6799fe97cab4a7865324d067'),
 'Asset Profile': {'Address1': '22, Avenue Montaigne',
  'City': 'Paris',
  'Zip': '75008',
  'Country': 'France',
  'Phone': '33 1 44 13 22 22',
  'Fax': '33 1 44 13 22 23',
  'Website': 'https://www.lvmh.com',
  'Industry': 'Luxury Goods',
  'Industry Key': 'luxury-goods',
  'Industry Disp': 'Luxury Goods',
  'Sector': 'Consumer Cyclical',
  'Sector Key': 'consumer-cyclical',
  'Sector Disp': 'Consumer Cyclical',
  'Long Business Summary': "LVMH Moët Hennessy - Louis Vuitton, Société Européenne operates as a luxury goods company worldwide. The company offers wines, and spirits under the domaine des Lambrays, Château d'Yquem, Dom Pérignon, Ruinart, Moët & Chandon, Hennessy, Veuve Clicquot, Château Galoupet, Ardbeg, Château Cheval Blanc, Glenmorangie, Krug, Mercier, Chandon, Joseph Phelps, Newton Vineyard, Cloudy Bay, Belvedere, Colgin Cellars,Terrazas de los Andes, Bodega Numanthia, Cheval des Andes, Woodinville, Ao Yun, Clos19, Volcan de 

## <u>Financials Module:</u>
In Financials.py, it collects financial metrics and follows the same logic as the Information module. The update frequency is set to 1 week by default, and the available keys are as follows:


In [25]:
dir(EQTYahoo.Financials)

['AnnualAccountsPayable',
 'AnnualAccountsReceivable',
 'AnnualAccruedInterestReceivable',
 'AnnualAccumulatedDepreciation',
 'AnnualAdditionalPaidInCapital',
 'AnnualAllowanceForDoubtfulAccountsReceivable',
 'AnnualAmortization',
 'AnnualAmortizationOfIntangiblesIncomeStatement',
 'AnnualAssetsHeldForSaleCurrent',
 'AnnualAvailableForSaleSecurities',
 'AnnualAverageDilutionEarnings',
 'AnnualBasicAccountingChange',
 'AnnualBasicAverageShares',
 'AnnualBasicContinuousOperations',
 'AnnualBasicDiscontinuousOperations',
 'AnnualBasicEPS',
 'AnnualBasicEPSOtherGainsLosses',
 'AnnualBasicExtraordinary',
 'AnnualBuildingsAndImprovements',
 'AnnualCapitalLeaseObligations',
 'AnnualCapitalStock',
 'AnnualCashAndCashEquivalents',
 'AnnualCashCashEquivalentsAndShortTermInvestments',
 'AnnualCashEquivalents',
 'AnnualCashFinancial',
 'AnnualCommercialPaper',
 'AnnualCommonStock',
 'AnnualCommonStockEquity',
 'AnnualConstructionInProgress',
 'AnnualContinuingAndDiscontinuedBasicEPS',
 'AnnualCont

You can access all financial information for a ticker using the `Financials` function.


In [26]:
EQTYahoo.Financials.Financials('AAPL')

{'Annual Accounts Payable': [{'Data ID': 23000,
   'As Of Date': '2021-09-30',
   'Period Type': '12M',
   'Currency': 'USD',
   'Value': 54763000000.0,
   'Formatted Value': '54.76B'},
  {'Data ID': 23000,
   'As Of Date': '2022-09-30',
   'Period Type': '12M',
   'Currency': 'USD',
   'Value': 64115000000.0,
   'Formatted Value': '64.11B'},
  {'Data ID': 23000,
   'As Of Date': '2023-09-30',
   'Period Type': '12M',
   'Currency': 'USD',
   'Value': 62611000000.0,
   'Formatted Value': '62.61B'},
  {'Data ID': 23000,
   'As Of Date': '2024-09-30',
   'Period Type': '12M',
   'Currency': 'USD',
   'Value': 68960000000.0,
   'Formatted Value': '68.96B'}],
 'Annual Accounts Receivable': [{'Data ID': 23001,
   'As Of Date': '2021-09-30',
   'Period Type': '12M',
   'Currency': 'USD',
   'Value': 26278000000.0,
   'Formatted Value': '26.28B'},
  {'Data ID': 23001,
   'As Of Date': '2022-09-30',
   'Period Type': '12M',
   'Currency': 'USD',
   'Value': 28184000000.0,
   'Formatted Value':

You can access a specific key by using the built-in functions of the `Financials` module and convert it into a DataFrame for better readability.


In [29]:
pandas.DataFrame(EQTYahoo.Financials.AnnualNetIncome('AAPL'))

,Data ID,As Of Date,Period Type,Currency,Value,Formatted Value
0,20091,2021-09-30,12M,USD,9.468000e+10,94.68B
1,20091,2022-09-30,12M,USD,9.980300e+10,99.80B
2,20091,2023-09-30,12M,USD,9.699500e+10,97.00B
3,20091,2024-09-30,12M,USD,9.373600e+10,93.74B


## <u>Options Module</u>:

Found in Options.py, retrieves option chains using the `Chain` function and supports filtering. Default parameters are as follows:

````
Ticker: str,
ContractsType: str = None,
StrikeRange: list[Unknown] = None,
MoneynessRange: list[Unknown] = None,
OpenInterestRange: list[Unknown] = None,
VolumeRange: list[Unknown] = None,
LastPriceRange: list[Unknown] = None,
ThirdFridaysOnly: bool = False
````

Between 9:30 AM and 4:30 PM, updates occur every 15 minutes; otherwise, updates occur every 24 hours. 

In [39]:
EQTYahoo.Options.Chain('TSLA')

TSLA Options Chain -- Last updated: 2025-01-29 11:44:07.662703


,_id,Contract Symbol,Contract Size,Last Trade Date,Underlying Name,Underlying Region,Underlying Ticker,Underlying Volume,Underlying Open Price,Underlying High Price,Underlying Low Price,Underlying Price,Underlying Currency,Underlying Exchange,Underlying Type,Underlying Quote Source,Underlying Dividend Yield,Contract Strike,Contract Type,Contract Expiration,Contract Last Price,Contract Open Interest,Contract Volume,Contract Bid,Contract Ask,Contract Change,Contract Percent Change,Contract Currency,Moneyness Formula,Contract Moneyness,Last Update,Third Friday
0,679a0677cab4a7865324d068,TSLA250131C00075000,REGULAR,2025-01-28 16:25:19,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,75.0,Call,2025-01-31,317.70,0,1.0,0.0,0.0,0.0,0.0,USD,(S/K),5.307867,2025-01-29 11:44:07.642712,False
1,679a0677cab4a7865324d069,TSLA250131C00080000,REGULAR,2024-12-31 15:39:30,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,80.0,Call,2025-01-31,336.35,0,NaN,0.0,0.0,0.0,0.0,USD,(S/K),4.976125,2025-01-29 11:44:07.642712,False
2,679a0677cab4a7865324d06a,TSLA250131C00090000,REGULAR,2025-01-23 20:48:48,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,90.0,Call,2025-01-31,320.45,0,1.0,0.0,0.0,0.0,0.0,USD,(S/K),4.423222,2025-01-29 11:44:07.642712,False
3,679a0677cab4a7865324d06b,TSLA250131C00100000,REGULAR,2025-01-27 16:01:47,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,100.0,Call,2025-01-31,302.15,0,2.0,0.0,0.0,0.0,0.0,USD,(S/K),3.980900,2025-01-29 11:44:07.642712,False
4,679a0677cab4a7865324d06c,TSLA250131C00105000,REGULAR,2025-01-27 16:14:03,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,105.0,Call,2025-01-31,296.10,0,2.0,0.0,0.0,0.0,0.0,USD,(S/K),3.791333,2025-01-29 11:44:07.642712,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4517,679a068fcab4a7865324e20d,TSLA270115P00900000,REGULAR,2025-01-27 16:44:44,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,900.0,Put,2027-01-15,507.00,0,1.0,0.0,0.0,0.0,0.0,USD,(K/S),2.260795,2025-01-29 11:44:07.662703,True
4518,679a068fcab4a7865324e20e,TSLA270115P00910000,REGULAR,2025-01-21 21:50:26,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,910.0,Put,2027-01-15,506.90,0,2.0,0.0,0.0,0.0,0.0,USD,(K/S),2.285915,2025-01-29 11:44:07.662703,True
4519,679a068fcab4a7865324e20f,TSLA270115P00930000,REGULAR,2025-01-17 16:19:51,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,930.0,Put,2027-01-15,516.40,0,4.0,0.0,0.0,0.0,0.0,USD,(K/S),2.336155,2025-01-29 11:44:07.662703,True
4520,679a068fcab4a7865324e210,TSLA270115P00950000,REGULAR,2025-01-27 19:38:37,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,950.0,Put,2027-01-15,561.98,0,1.0,0.0,0.0,0.0,0.0,USD,(K/S),2.386395,2025-01-29 11:44:07.662703,True


For example, if you're only looking for ATM calls expiring on the third Friday of the month:

In [40]:
EQTYahoo.Options.Chain(Ticker='TSLA', MoneynessRange=[0.9, 1.1], ThirdFridaysOnly=True)

TSLA Options Chain -- Last updated: 2025-01-29 11:44:07.662703


,_id,Contract Symbol,Contract Size,Last Trade Date,Underlying Name,Underlying Region,Underlying Ticker,Underlying Volume,Underlying Open Price,Underlying High Price,Underlying Low Price,Underlying Price,Underlying Currency,Underlying Exchange,Underlying Type,Underlying Quote Source,Underlying Dividend Yield,Contract Strike,Contract Type,Contract Expiration,Contract Last Price,Contract Open Interest,Contract Volume,Contract Bid,Contract Ask,Contract Change,Contract Percent Change,Contract Currency,Moneyness Formula,Contract Moneyness,Last Update,Third Friday
833,679a0679cab4a7865324d3a9,TSLA250221C00365000,REGULAR,2025-01-28 21:22:25,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,365.0,Call,2025-02-21,48.00,0,276.0,0.0,0.0,0.0,0.0,USD,(S/K),1.090658,2025-01-29 11:44:07.646707,True
834,679a0679cab4a7865324d3aa,TSLA250221C00370000,REGULAR,2025-01-28 21:55:00,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,370.0,Call,2025-02-21,43.70,0,837.0,0.0,0.0,0.0,0.0,USD,(S/K),1.075919,2025-01-29 11:44:07.646707,True
835,679a0679cab4a7865324d3ab,TSLA250221C00375000,REGULAR,2025-01-28 21:57:09,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,375.0,Call,2025-02-21,40.15,0,327.0,0.0,0.0,0.0,0.0,USD,(S/K),1.061573,2025-01-29 11:44:07.646707,True
836,679a0679cab4a7865324d3ac,TSLA250221C00380000,REGULAR,2025-01-28 21:59:34,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,380.0,Call,2025-02-21,37.70,0,1610.0,0.0,0.0,0.0,0.0,USD,(S/K),1.047605,2025-01-29 11:44:07.646707,True
837,679a0679cab4a7865324d3ad,TSLA250221C00385000,REGULAR,2025-01-28 21:59:55,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,385.0,Call,2025-02-21,34.50,0,521.0,0.0,0.0,0.0,0.0,USD,(S/K),1.034000,2025-01-29 11:44:07.646707,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4466,679a068ecab4a7865324e1da,TSLA270115P00390000,REGULAR,2025-01-28 21:31:54,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,390.0,Put,2027-01-15,103.00,0,14.0,0.0,0.0,0.0,0.0,USD,(K/S),0.979678,2025-01-29 11:44:07.662703,True
4467,679a068ecab4a7865324e1db,TSLA270115P00400000,REGULAR,2025-01-28 19:34:31,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,400.0,Put,2027-01-15,110.45,0,18.0,0.0,0.0,0.0,0.0,USD,(K/S),1.004798,2025-01-29 11:44:07.662703,True
4468,679a068ecab4a7865324e1dc,TSLA270115P00410000,REGULAR,2025-01-28 21:57:24,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,410.0,Put,2027-01-15,115.68,0,32.0,0.0,0.0,0.0,0.0,USD,(K/S),1.029918,2025-01-29 11:44:07.662703,True
4469,679a068ecab4a7865324e1dd,TSLA270115P00420000,REGULAR,2025-01-27 21:45:37,"Tesla, Inc.",US,TSLA,48417586,396.91,400.59,386.5,398.09,USD,NasdaqGS,Equity,Nasdaq Real Time Price,None,420.0,Put,2027-01-15,122.75,0,13.0,0.0,0.0,0.0,0.0,USD,(K/S),1.055038,2025-01-29 11:44:07.662703,True
